In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [24]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk
import datetime

## User Input

In [67]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver


## Scraping

In [68]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

form_data = {"over18": "yes"}

response = requests.get(user_url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("Over 18, bypassing...")
    age_url =soup.select_one('link[rel="canonical"]')['href']
    new_response = requests.post(age_url, data=form_data, headers=headers)
    soup = BeautifulSoup(new_response.content, 'html.parser')
    title_check = soup.title.text.split(' ')[0]
    if title_check == "overview":
        print("bypass working")
    else:
        print("bypass failed")

Over 18, bypassing...
bypass working


### Profile Info

In [69]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''


user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0

print(username)
print(comment_karma)
print(post_karma)
print(user_creation_date)
print(num_posts)
print(num_comments)

AmyOcean
12
26
2023-05-13T02:50:33+00:00
0
0


### Comments

In [94]:
comments = []
comment_count = 0
spans = []
date_format = "%Y-%m-%dT%H:%M:%S%z"

for comment in soup.find_all('div', class_='comment'):
    utc_date = comment.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    spans = comment.find('span', class_='score unvoted')
    if spans is not None:
        score = spans.text
    subreddit = comment.find('a', class_="subreddit").text
    permalink = comment.find('a', class_="bylink")['href']
    text = comment.find('div', class_='md-container').text
    comment_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'text': text
    }
    comments.append(comment_data)
    comment_count += 1


print(comments)
print(comment_count)

[{'utc_date': '2023-05-14T13:33:15+00:00', 'score': '1 point', 'subreddit': 'ShingekiNoKyojin', 'permalink': 'https://old.reddit.com/r/ShingekiNoKyojin/comments/13h9bnz/what_to_watch_after_attack_on_titan/jk45qe8/', 'text': 'My favorite after attack on Titan was To your Eternity . Fate Stay Night Unlimited blades is another one to go to right after. I think Full Metal Alchemist Brotherhood is also a good option. \nBut in my opinion, nothing compares to Attack on Titan\n\n'}, {'utc_date': '2023-05-14T13:02:35+00:00', 'score': '1 point', 'subreddit': 'KimetsuNoYaiba', 'permalink': 'https://old.reddit.com/r/KimetsuNoYaiba/comments/13g8i4n/nezuko_cosplay_self/jk42dkv/', 'text': 'Perfectly done!\n\n'}, {'utc_date': '2023-05-14T13:00:22+00:00', 'score': '1 point', 'subreddit': 'Animemes', 'permalink': 'https://old.reddit.com/r/Animemes/comments/13h9wvg/thighdeology/jk42558/', 'text': 'So cute!\n\n'}, {'utc_date': '2023-05-14T02:18:05+00:00', 'score': '1 point', 'subreddit': 'CelebBattles', '

### Posts

In [95]:
#Counting Posts

post_count = 0
for post in soup.find_all('div', class_='link'):
    print(post)
    post_count += 1
print(post_count)

<div class="thing id-t3_13hdchh over18 odd link self" data-author="AmyOcean" data-author-fullname="t2_nahg6uuv" data-comments-count="0" data-context="listing" data-domain="self.eroticliterature" data-fullname="t3_13hdchh" data-gildings="0" data-is-gallery="false" data-nsfw="true" data-num-crossposts="0" data-oc="false" data-permalink="/r/eroticliterature/comments/13hdchh/amys_lesbian_diary_ff/" data-promoted="false" data-rank="" data-score="1" data-spoiler="false" data-subreddit="eroticliterature" data-subreddit-fullname="t5_2rsj3" data-subreddit-prefixed="r/eroticliterature" data-subreddit-type="public" data-timestamp="1684073683000" data-type="link" data-url="/r/eroticliterature/comments/13hdchh/amys_lesbian_diary_ff/" data-whitelist-status="" id="thing_t3_13hdchh" onclick="click_thing(this)"><p class="parent"></p><span class="rank"></span><div class="midcol unvoted"><div aria-label="upvote" class="arrow up login-required access-required" data-event-action="upvote" role="button" tabi

In [96]:
#Identify Posts

posts = []
post_count = 0

utc_date = 0
post_type = ""
subreddit = ""
post_title = ""
file_path = ""
permalink = ""
score = 0
text = ""

#self post
for self_post in soup.find_all('div', class_='self'):
    post_count += 1
    utc_date = self_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "self"
    subreddit = self_post.find('a', class_="subreddit").text
    post_title = self_post.find('a', class_="title").text
    score = self_post.find('div', {'class': 'score unvoted'}).text
    permalink = self_post.find('a', class_="bylink")['href']

    if self_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        print("self expando")
        form_data = {"over18": "yes"}
        response = requests.get(permalink, data=form_data, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        print(post_soup)
        text = post_soup.find('div', {'class': 'entry'}).find('div', {'class': 'md-container'}).get_text()
        
    else:
        text = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
    
#crosspost
for cross_post in soup.find_all('div', {'class':'link', 'data-crosspost-root-title': True}):
    post_count += 1
    utc_date = cross_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "crosspost"
    subreddit = cross_post.find('a', class_="subreddit").text
    post_title = cross_post.find('a', class_="title").text
    score = cross_post.find('div', {'class': 'score unvoted'}).text
    permalink = cross_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#link
domains = ['old.reddit.com', 'i.redd.it', 'v.redd.it', 'i.imgur.com']
for link_post in soup.find_all(lambda tag: tag.has_attr('data-domain') and not any(domain in tag['data-domain'] for domain in domains )and 'self' not in tag.get('class', [])):
    post_count += 1
    utc_date = link_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "link"
    subreddit = link_post.find('a', class_="subreddit").text
    post_title = link_post.find('a', class_="title").text
    score = link_post.find('div', {'class': 'score unvoted'}).text
    permalink = link_post.find('a', class_="bylink")['href']
    
    #If expando, request permalink page, extract text, else text = NaN
    
    if link_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        form_data = {"over18": "yes"}
        response = requests.get(permalink, data=form_data, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        text = post_soup.find('div', {'class': 'entry'}).find('div', {'class': 'md-container'}).get_text()
    else:
        text = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#video
for video_post in soup.find_all('div', {'class':'link', "data-domain": "v.redd.it"}):
    post_count += 1
    utc_date = video_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "video"
    subreddit = video_post.find('a', class_="subreddit").text
    post_title = video_post.find('a', class_="title").text
    score = video_post.find('div', {'class': 'score unvoted'}).text
    permalink = video_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#picture 
for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.redd.it"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.imgur.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
print(post_count)

self expando
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><title>reddit.com: over 18?</title><meta content=" reddit, reddit.com, vote, comment, submit " name="keywords"/><meta content="Reddit gives you the best of the internet in one place. Get a constantly updating feed of breaking news, fun stories, pics, memes, and videos just for you. Passionate about something niche? Reddit has thousands of vibrant communities with people that share your interests. Alternatively, find out what’s trending across all of Reddit on r/popular. Reddit is also anonymous so you can be yourself, with your Reddit profile and persona disconnected from your real-world identity." name="description"/><meta content="origin-when-cross-origin" name="referrer"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"><link href="/static/opensearch.xml" rel="search" type="application/opensearchdescription+xml"/><link href="https://www.reddit.com/over18/?dest=http

AttributeError: 'NoneType' object has no attribute 'find'

In [62]:
#Test if 25 Posts identified (Number of each page)
if (post_count + comment_count) != 25:
    print("Error")
else:
    print("working")

working


In [63]:
num_posts = len(posts)
num_comments = len(comments)

In [64]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [65]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

In [66]:
with open('Temp_Files/user_data.json') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "user_info": {
        "username": "NPU-F",
        "user_creation_date": "2015-11-16T22:40:48+00:00",
        "num_posts": 11,
        "num_comments": 14,
        "karma": {
            "post_karma": "28,285",
            "comment_karma": "15,020"
        }
    },
    "posts": [
        {
            "utc_date": "2023-05-12T11:46:07+00:00",
            "score": "222",
            "subreddit": "r/Atlanta",
            "permalink": "https://old.reddit.com/r/Atlanta/comments/13fitgj/vice_president_kamala_harris_visiting_atlanta/",
            "title": "Vice President Kamala Harris visiting Atlanta Friday for DNC events",
            "type": "link",
            "text": null
        },
        {
            "utc_date": "2023-05-11T13:41:41+00:00",
            "score": "579",
            "subreddit": "r/Atlanta",
            "permalink": "https://old.reddit.com/r/Atlanta/comments/13enrrk/plans_to_connect_silver_comet_trail_atlanta/",
            "title": "Plans to connect Silver Comet

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* blank

**Minor Additions**
* Accounting for pinned posts
* Profile pictures (snoovatar??)
* Trophies
* Add over 18 bypass for old.reddit